# 🏦 BVMT Stock Price Prediction with XGBoost
## IHEC-CODELAB 2.0 - Complete Production-Ready Pipeline

**Objective:** Predict the next 5 days closing prices for BVMT stocks

**Model:** XGBoost (fast, accurate, interpretable)

**Metrics:** RMSE, MAE, Directional Accuracy (most important for trading)

---

### 📋 Table of Contents
1. Setup & Data Loading
2. Data Exploration & Cleaning
3. Feature Engineering
4. Model Training with Optimized Hyperparameters
5. Evaluation & Visualization
6. Export for Production

## 1. 📦 Setup & Data Loading

In [ ]:
# Install required packages (run in Colab)
!pip install -q xgboost pandas numpy scikit-learn matplotlib seaborn ta joblib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print("✅ Libraries imported successfully!")

In [ ]:
# 📁 Upload your data files to Colab
# Run this cell and upload all histo_cotation files
from google.colab import files
uploaded = files.upload()
print(f"✅ Uploaded {len(uploaded)} files")

In [ ]:
def load_txt_file(filepath):
    """
    Load 2016-2021 fixed-width text files.
    These have a different format than the CSV files.
    """
    # Read with fixed width format
    df = pd.read_fwf(
        filepath,
        skiprows=2,  # Skip header rows
        header=None,
        names=['SEANCE', 'GROUPE', 'CODE', 'VALEUR', 'OUVERTURE', 'CLOTURE', 
               'PLUS_BAS', 'PLUS_HAUT', 'QUANTITE_NEGOCIEE', 'NB_TRANSACTION', 'CAPITAUX', 'IND_RES'],
        encoding='latin-1'
    )
    # Drop the IND_RES column if it exists
    if 'IND_RES' in df.columns:
        df = df.drop('IND_RES', axis=1)
    return df

def load_csv_file(filepath):
    """
    Load 2022-2025 CSV files (semicolon-separated).
    """
    df = pd.read_csv(filepath, sep=';', encoding='utf-8')
    # Clean column names
    df.columns = df.columns.str.strip()
    return df

def load_all_data():
    """
    Load all available data files and combine them.
    """
    all_dfs = []
    
    # Load TXT files (2016-2021)
    for year in range(2016, 2022):
        try:
            df = load_txt_file(f'histo_cotation_{year}.txt')
            df['source_year'] = year
            all_dfs.append(df)
            print(f"✅ Loaded {year}: {len(df)} rows")
        except Exception as e:
            print(f"⚠️ Could not load {year}: {e}")
    
    # Load CSV files (2022-2025)
    for year in range(2022, 2026):
        try:
            df = load_csv_file(f'histo_cotation_{year}.csv')
            df['source_year'] = year
            all_dfs.append(df)
            print(f"✅ Loaded {year}: {len(df)} rows")
        except Exception as e:
            print(f"⚠️ Could not load {year}: {e}")
    
    # Combine all dataframes
    combined = pd.concat(all_dfs, ignore_index=True)
    print(f"\n📊 Total combined: {len(combined)} rows")
    return combined

# Load all data
raw_data = load_all_data()

## 2. 🔍 Data Exploration & Cleaning

In [ ]:
# Display basic info
print("📋 Dataset Info:")
print(f"Shape: {raw_data.shape}")
print(f"\nColumns: {raw_data.columns.tolist()}")
print(f"\nData types:\n{raw_data.dtypes}")
print(f"\nSample data:")
raw_data.head(10)

In [ ]:
def clean_data(df):
    """
    Clean and standardize the BVMT dataset.
    
    Steps:
    1. Parse dates
    2. Filter to main market (GROUPE=11)
    3. Convert numeric columns
    4. Handle missing values
    5. Remove zero-volume days (no trading)
    """
    df = df.copy()
    
    # 1. Parse date - handle multiple formats
    def parse_date(x):
        if pd.isna(x):
            return pd.NaT
        x = str(x).strip()
        for fmt in ['%d/%m/%Y', '%Y-%m-%d', '%d-%m-%Y']:
            try:
                return pd.to_datetime(x, format=fmt)
            except:
                continue
        return pd.NaT
    
    df['date'] = df['SEANCE'].apply(parse_date)
    
    # 2. Filter to main market stocks (GROUPE=11)
    df['GROUPE'] = pd.to_numeric(df['GROUPE'], errors='coerce')
    df = df[df['GROUPE'] == 11].copy()
    print(f"✅ Filtered to main market: {len(df)} rows")
    
    # 3. Convert numeric columns
    numeric_cols = ['OUVERTURE', 'CLOTURE', 'PLUS_BAS', 'PLUS_HAUT', 
                    'QUANTITE_NEGOCIEE', 'NB_TRANSACTION', 'CAPITAUX']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Rename columns to English
    df = df.rename(columns={
        'VALEUR': 'stock_name',
        'CODE': 'stock_code',
        'OUVERTURE': 'open',
        'CLOTURE': 'close',
        'PLUS_BAS': 'low',
        'PLUS_HAUT': 'high',
        'QUANTITE_NEGOCIEE': 'volume',
        'NB_TRANSACTION': 'transactions',
        'CAPITAUX': 'capital'
    })
    
    # 4. Handle missing/invalid values
    # Remove rows with NaN dates
    df = df.dropna(subset=['date'])
    
    # Remove rows with zero or NaN close price
    df = df[(df['close'] > 0) & df['close'].notna()]
    
    # 5. Remove zero-volume days (no actual trading)
    df = df[df['volume'] > 0].copy()
    print(f"✅ After removing zero-volume days: {len(df)} rows")
    
    # Sort by stock and date
    df = df.sort_values(['stock_name', 'date']).reset_index(drop=True)
    
    return df

# Clean the data
data = clean_data(raw_data)
print(f"\n📊 Cleaned dataset shape: {data.shape}")
data.head()

In [ ]:
# Analyze stocks - focus on most liquid ones
stock_stats = data.groupby('stock_name').agg({
    'volume': ['count', 'mean', 'sum'],
    'close': ['mean', 'std'],
    'date': ['min', 'max']
}).round(2)

stock_stats.columns = ['trading_days', 'avg_volume', 'total_volume', 'avg_price', 'price_std', 'first_date', 'last_date']
stock_stats = stock_stats.sort_values('total_volume', ascending=False)

print("🏆 Top 15 Most Liquid Stocks (by total volume):")
top_stocks = stock_stats.head(15)
display(top_stocks)

In [ ]:
# Select top liquid stocks for modeling
TOP_N_STOCKS = 15
selected_stocks = stock_stats.head(TOP_N_STOCKS).index.tolist()

# Filter data to selected stocks
data = data[data['stock_name'].isin(selected_stocks)].copy()
print(f"\n📊 Selected {TOP_N_STOCKS} stocks with {len(data)} total trading records")
print(f"\nSelected stocks: {selected_stocks}")

In [ ]:
# Visualize price history for top stocks
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes = axes.flatten()

for i, stock in enumerate(selected_stocks[:6]):
    stock_data = data[data['stock_name'] == stock]
    axes[i].plot(stock_data['date'], stock_data['close'], color='#ff4444', linewidth=1)
    axes[i].set_title(f'{stock}', fontsize=12, color='white')
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Close Price (TND)')
    axes[i].grid(alpha=0.3)

plt.suptitle('BVMT Top Stocks - Price History', fontsize=16, color='white', y=1.02)
plt.tight_layout()
plt.show()

## 3. 🛠️ Feature Engineering

This is the **most critical step** for XGBoost performance.

We'll create:
- **Price features**: Returns, volatility, moving averages
- **Volume features**: Volume ratios, trends
- **Technical indicators**: RSI, MACD, Bollinger Bands
- **Time features**: Day of week, month effects
- **Lagged features**: Past N days prices

In [ ]:
def create_features(df, lookback=60):
    """
    Create comprehensive features for stock prediction.
    
    Args:
        df: DataFrame with OHLCV data for a single stock
        lookback: Number of past days to use for features
    
    Returns:
        DataFrame with all features
    """
    df = df.copy().sort_values('date').reset_index(drop=True)
    
    # ===== PRICE FEATURES =====
    
    # Returns (log returns for stability)
    df['return_1d'] = np.log(df['close'] / df['close'].shift(1))
    df['return_5d'] = np.log(df['close'] / df['close'].shift(5))
    df['return_10d'] = np.log(df['close'] / df['close'].shift(10))
    df['return_20d'] = np.log(df['close'] / df['close'].shift(20))
    
    # Volatility (rolling std of returns)
    df['volatility_5d'] = df['return_1d'].rolling(5).std()
    df['volatility_10d'] = df['return_1d'].rolling(10).std()
    df['volatility_20d'] = df['return_1d'].rolling(20).std()
    
    # Moving Averages
    for window in [5, 10, 20, 50]:
        df[f'sma_{window}'] = df['close'].rolling(window).mean()
        df[f'price_to_sma_{window}'] = df['close'] / df[f'sma_{window}']
    
    # Exponential Moving Averages
    df['ema_12'] = df['close'].ewm(span=12).mean()
    df['ema_26'] = df['close'].ewm(span=26).mean()
    df['price_to_ema_12'] = df['close'] / df['ema_12']
    
    # ===== VOLUME FEATURES =====
    
    df['volume_sma_5'] = df['volume'].rolling(5).mean()
    df['volume_sma_20'] = df['volume'].rolling(20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_sma_20']
    df['volume_change'] = df['volume'].pct_change()
    
    # ===== TECHNICAL INDICATORS =====
    
    # RSI (Relative Strength Index)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    # MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_middle'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_middle'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    # Price Range Features
    df['intraday_range'] = (df['high'] - df['low']) / df['close']
    df['gap_open'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)
    
    # ===== TIME FEATURES =====
    
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    
    # ===== LAGGED FEATURES =====
    
    # Past N days closing prices (normalized as returns)
    for lag in [1, 2, 3, 5, 10, 20]:
        df[f'lag_{lag}d_return'] = np.log(df['close'] / df['close'].shift(lag))
    
    return df

# Apply feature engineering to each stock
print("🛠️ Creating features for each stock...")
featured_data = []

for stock in selected_stocks:
    stock_df = data[data['stock_name'] == stock].copy()
    stock_df = create_features(stock_df)
    featured_data.append(stock_df)
    print(f"  ✅ {stock}: {len(stock_df)} rows")

data_with_features = pd.concat(featured_data, ignore_index=True)
print(f"\n📊 Total feature-enriched data: {data_with_features.shape}")

In [ ]:
# Display feature columns
feature_cols = [col for col in data_with_features.columns if col not in 
                ['date', 'stock_name', 'stock_code', 'GROUPE', 'source_year', 
                 'open', 'high', 'low', 'close', 'volume', 'transactions', 'capital']]

print(f"📋 Created {len(feature_cols)} features:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

## 4. 🎯 Create Target Variables (5-Day Prediction)

In [ ]:
def create_targets(df, horizons=[1, 2, 3, 4, 5]):
    """
    Create target variables for multi-horizon prediction.
    
    We predict:
    - Future returns (for direction)
    - Future prices (for actual value)
    """
    df = df.copy()
    
    for h in horizons:
        # Target: Price return in h days
        df[f'target_return_{h}d'] = (df['close'].shift(-h) - df['close']) / df['close']
        
        # Target: Actual future price
        df[f'target_price_{h}d'] = df['close'].shift(-h)
        
        # Target: Direction (1=up, 0=down)
        df[f'target_direction_{h}d'] = (df['close'].shift(-h) > df['close']).astype(int)
    
    return df

# Apply to each stock separately
print("🎯 Creating target variables...")
final_data = []

for stock in selected_stocks:
    stock_df = data_with_features[data_with_features['stock_name'] == stock].copy()
    stock_df = create_targets(stock_df)
    final_data.append(stock_df)

df_final = pd.concat(final_data, ignore_index=True)

# Drop rows with NaN features or targets
initial_rows = len(df_final)
df_final = df_final.dropna()
print(f"✅ Dropped {initial_rows - len(df_final)} rows with NaN values")
print(f"📊 Final dataset: {df_final.shape}")

In [ ]:
# Define feature columns for training
FEATURE_COLS = [
    # Returns
    'return_1d', 'return_5d', 'return_10d', 'return_20d',
    # Volatility
    'volatility_5d', 'volatility_10d', 'volatility_20d',
    # Moving Averages
    'price_to_sma_5', 'price_to_sma_10', 'price_to_sma_20', 'price_to_sma_50',
    'price_to_ema_12',
    # Volume
    'volume_ratio', 'volume_change',
    # Technical Indicators
    'rsi_14', 'macd', 'macd_signal', 'macd_hist',
    'bb_width', 'bb_position',
    # Price Range
    'intraday_range', 'gap_open',
    # Time
    'day_of_week', 'month', 'quarter', 'is_month_start', 'is_month_end',
    # Lagged Returns
    'lag_1d_return', 'lag_2d_return', 'lag_3d_return', 'lag_5d_return', 'lag_10d_return', 'lag_20d_return'
]

# Verify all features exist
missing_features = [f for f in FEATURE_COLS if f not in df_final.columns]
if missing_features:
    print(f"⚠️ Missing features: {missing_features}")
else:
    print(f"✅ All {len(FEATURE_COLS)} features available")

## 5. 🚂 Model Training with Optimized Hyperparameters

### XGBoost Hyperparameters (Optimized for Stock Prediction)

Based on winning solutions from Kaggle stock prediction competitions and research papers:

- **n_estimators**: 500-1000 (with early stopping)
- **max_depth**: 6-8 (prevent overfitting on noisy data)
- **learning_rate**: 0.01-0.05 (slow and steady)
- **subsample**: 0.8 (bootstrap samples)
- **colsample_bytree**: 0.8 (feature sampling)
- **min_child_weight**: 5-10 (regularization)
- **reg_alpha/reg_lambda**: L1/L2 regularization

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb

# Prepare training data
# Use temporal split: Train on older data, test on recent data

# Sort by date
df_sorted = df_final.sort_values('date').reset_index(drop=True)

# Split: 80% train, 10% validation, 10% test (temporal)
n = len(df_sorted)
train_end = int(n * 0.8)
val_end = int(n * 0.9)

train_df = df_sorted.iloc[:train_end]
val_df = df_sorted.iloc[train_end:val_end]
test_df = df_sorted.iloc[val_end:]

print(f"📊 Data Split:")
print(f"  Train: {len(train_df)} rows ({train_df['date'].min()} to {train_df['date'].max()})")
print(f"  Val:   {len(val_df)} rows ({val_df['date'].min()} to {val_df['date'].max()})")
print(f"  Test:  {len(test_df)} rows ({test_df['date'].min()} to {test_df['date'].max()})")

In [ ]:
# Prepare features and targets
X_train = train_df[FEATURE_COLS].values
X_val = val_df[FEATURE_COLS].values
X_test = test_df[FEATURE_COLS].values

# We'll train separate models for each prediction horizon (1-5 days)
# For hackathon, we'll focus on predicting returns and convert to prices

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Features scaled")
print(f"  X_train: {X_train_scaled.shape}")
print(f"  X_val: {X_val_scaled.shape}")
print(f"  X_test: {X_test_scaled.shape}")

In [ ]:
# Optimized XGBoost parameters for stock prediction
XGB_PARAMS = {
    'objective': 'reg:squarederror',
    'n_estimators': 1000,
    'max_depth': 6,
    'learning_rate': 0.03,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 5,
    'reg_alpha': 0.1,       # L1 regularization
    'reg_lambda': 1.0,      # L2 regularization
    'random_state': 42,
    'n_jobs': -1,
    'early_stopping_rounds': 50,
}

print("🎛️ XGBoost Hyperparameters:")
for k, v in XGB_PARAMS.items():
    print(f"  {k}: {v}")

In [ ]:
# Train models for each prediction horizon
HORIZONS = [1, 2, 3, 4, 5]
models = {}
results = {}

for horizon in HORIZONS:
    print(f"\n🚂 Training model for {horizon}-day prediction...")
    
    # Get targets
    y_train = train_df[f'target_return_{horizon}d'].values
    y_val = val_df[f'target_return_{horizon}d'].values
    y_test = test_df[f'target_return_{horizon}d'].values
    
    # Create and train model
    model = xgb.XGBRegressor(**XGB_PARAMS)
    
    model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_val_scaled, y_val)],
        verbose=False
    )
    
    # Predictions
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    train_mae = mean_absolute_error(y_train, y_pred_train)
    val_mae = mean_absolute_error(y_val, y_pred_val)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Directional accuracy (most important for trading!)
    train_dir_acc = np.mean((y_pred_train > 0) == (y_train > 0))
    val_dir_acc = np.mean((y_pred_val > 0) == (y_val > 0))
    test_dir_acc = np.mean((y_pred_test > 0) == (y_test > 0))
    
    # Store model and results
    models[horizon] = model
    results[horizon] = {
        'train_rmse': train_rmse, 'val_rmse': val_rmse, 'test_rmse': test_rmse,
        'train_mae': train_mae, 'val_mae': val_mae, 'test_mae': test_mae,
        'train_dir_acc': train_dir_acc, 'val_dir_acc': val_dir_acc, 'test_dir_acc': test_dir_acc,
        'best_iteration': model.best_iteration if hasattr(model, 'best_iteration') else XGB_PARAMS['n_estimators']
    }
    
    print(f"  ✅ {horizon}d Model - Test RMSE: {test_rmse:.6f}, Test MAE: {test_mae:.6f}")
    print(f"     📈 Directional Accuracy: Train={train_dir_acc:.2%}, Val={val_dir_acc:.2%}, Test={test_dir_acc:.2%}")

In [ ]:
# Display results summary
results_df = pd.DataFrame(results).T
results_df.index.name = 'Horizon (days)'

print("\n📊 Model Performance Summary:")
print("="*80)
print(results_df.round(4).to_string())
print("="*80)

# Calculate average directional accuracy
avg_dir_acc = results_df['test_dir_acc'].mean()
print(f"\n🎯 Average Directional Accuracy: {avg_dir_acc:.2%}")
print("(Anything above 50% is profitable in theory!)")

In [ ]:
# Feature Importance Analysis
print("\n📊 Feature Importance (1-day model):")

importance = pd.DataFrame({
    'feature': FEATURE_COLS,
    'importance': models[1].feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 15 features
fig, ax = plt.subplots(figsize=(10, 8))
top_features = importance.head(15)
ax.barh(top_features['feature'], top_features['importance'], color='#ff4444')
ax.set_xlabel('Importance')
ax.set_title('Top 15 Most Important Features', color='white')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print(importance.to_string())

## 6. 📈 Prediction Visualization

In [ ]:
# Visualize predictions for a sample stock
sample_stock = 'SFBT'
sample_data = test_df[test_df['stock_name'] == sample_stock].copy()

if len(sample_data) > 0:
    # Get features for this stock
    X_sample = scaler.transform(sample_data[FEATURE_COLS].values)
    
    # Predict returns and convert to prices
    for horizon in HORIZONS:
        pred_returns = models[horizon].predict(X_sample)
        sample_data[f'pred_price_{horizon}d'] = sample_data['close'] * (1 + pred_returns)
    
    # Plot actual vs predicted
    fig, ax = plt.subplots(figsize=(14, 6))
    
    ax.plot(sample_data['date'], sample_data['close'], 
            label='Actual Price', color='white', linewidth=2)
    
    ax.plot(sample_data['date'], sample_data[f'target_price_5d'].shift(5), 
            label='Actual 5-day Forward', color='#00ff00', linewidth=1, linestyle='--', alpha=0.7)
    
    ax.plot(sample_data['date'], sample_data['pred_price_5d'], 
            label='Predicted 5-day Forward', color='#ff4444', linewidth=1, linestyle='--')
    
    ax.set_title(f'{sample_stock} - Price Prediction (Test Set)', color='white', fontsize=14)
    ax.set_xlabel('Date')
    ax.set_ylabel('Price (TND)')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️ No test data for {sample_stock}")

## 7. 💾 Export Models for Production

In [ ]:
import joblib
import json

# Create models directory
import os
os.makedirs('models', exist_ok=True)

# Save all models
for horizon, model in models.items():
    model.save_model(f'models/price_predictor_{horizon}d.json')
    print(f"✅ Saved models/price_predictor_{horizon}d.json")

# Save scaler
joblib.dump(scaler, 'models/feature_scaler.pkl')
print("✅ Saved models/feature_scaler.pkl")

# Save configuration
config = {
    'feature_columns': FEATURE_COLS,
    'prediction_horizons': HORIZONS,
    'selected_stocks': selected_stocks,
    'model_params': {k: v for k, v in XGB_PARAMS.items() if k != 'early_stopping_rounds'},
    'results': {str(k): {kk: float(vv) for kk, vv in v.items()} for k, v in results.items()},
    'created_at': datetime.now().isoformat()
}

with open('models/config.json', 'w') as f:
    json.dump(config, f, indent=2)
print("✅ Saved models/config.json")

print("\n📦 All models exported successfully!")

In [ ]:
# Download models (for Colab)
from google.colab import files

# Create a zip of all models
!zip -r models.zip models/
files.download('models.zip')

print("\n📥 Download complete! Extract models.zip to your project's ml/models/ folder")

## 8. 🔧 Inference Function (for Backend Integration)

Copy this function to your backend service!

In [ ]:
# === COPY THIS TO YOUR BACKEND ===

class BVMTPricePredictor:
    """
    Production-ready price prediction service for BVMT stocks.
    
    Usage:
        predictor = BVMTPricePredictor('models/')
        prediction = predictor.predict('SFBT', current_features)
    """
    
    def __init__(self, model_dir: str):
        self.model_dir = model_dir
        self.models = {}
        self.scaler = None
        self.config = None
        self._load_models()
    
    def _load_models(self):
        """Load all models and config"""
        import json
        import joblib
        import xgboost as xgb
        
        # Load config
        with open(f'{self.model_dir}/config.json', 'r') as f:
            self.config = json.load(f)
        
        # Load scaler
        self.scaler = joblib.load(f'{self.model_dir}/feature_scaler.pkl')
        
        # Load models for each horizon
        for horizon in self.config['prediction_horizons']:
            model = xgb.XGBRegressor()
            model.load_model(f'{self.model_dir}/price_predictor_{horizon}d.json')
            self.models[horizon] = model
        
        print(f"✅ Loaded {len(self.models)} models for horizons: {list(self.models.keys())}")
    
    def predict(self, stock_name: str, features: dict) -> dict:
        """
        Predict prices for next 5 days.
        
        Args:
            stock_name: Name of the stock (e.g., 'SFBT')
            features: Dictionary of feature values
        
        Returns:
            Dictionary with predictions for each horizon
        """
        import numpy as np
        
        # Get current price (must be in features)
        current_price = features.get('close', features.get('current_price'))
        if current_price is None:
            raise ValueError("'close' or 'current_price' must be in features")
        
        # Prepare feature vector
        feature_cols = self.config['feature_columns']
        X = np.array([[features.get(col, 0) for col in feature_cols]])
        X_scaled = self.scaler.transform(X)
        
        # Predict for each horizon
        predictions = {}
        for horizon in self.config['prediction_horizons']:
            pred_return = self.models[horizon].predict(X_scaled)[0]
            pred_price = current_price * (1 + pred_return)
            
            predictions[f'day_{horizon}'] = {
                'predicted_price': round(pred_price, 3),
                'predicted_return': round(pred_return * 100, 2),  # As percentage
                'direction': 'UP' if pred_return > 0 else 'DOWN',
                'confidence': round(abs(pred_return) * 10, 2)  # Simple confidence proxy
            }
        
        # Overall recommendation
        avg_return = np.mean([predictions[f'day_{h}']['predicted_return'] for h in [1,2,3,4,5]])
        if avg_return > 1.5:
            recommendation = 'BUY'
        elif avg_return < -1.5:
            recommendation = 'SELL'
        else:
            recommendation = 'HOLD'
        
        return {
            'stock': stock_name,
            'current_price': current_price,
            'predictions': predictions,
            'recommendation': recommendation,
            'avg_5d_return': round(avg_return, 2)
        }

# Test the predictor
print("\n🧪 Testing BVMTPricePredictor...")

# Create sample features (use last row from test set)
sample_features = dict(zip(FEATURE_COLS, X_test_scaled[-1]))
sample_features['close'] = test_df['close'].iloc[-1]

# This would work with saved models:
# predictor = BVMTPricePredictor('models/')
# result = predictor.predict('SFBT', sample_features)
# print(json.dumps(result, indent=2))

print("✅ Inference function ready for backend integration!")

## 9. 📋 Summary & Next Steps

### What We Built:
1. ✅ Data loading pipeline for BVMT historical data (2016-2025)
2. ✅ Comprehensive feature engineering (30+ features)
3. ✅ XGBoost models for 1-5 day price prediction
4. ✅ Directional accuracy metrics for trading decisions
5. ✅ Production-ready inference class

### Model Performance:
- Check the results summary above
- Directional accuracy > 50% = potentially profitable
- Focus on directional accuracy in your pitch!

### Files to Copy to Your Project:
```
ml/models/
├── price_predictor_1d.json
├── price_predictor_2d.json
├── price_predictor_3d.json
├── price_predictor_4d.json
├── price_predictor_5d.json
├── feature_scaler.pkl
└── config.json
```

### Backend Integration:
Copy the `BVMTPricePredictor` class to `backend/app/services/prediction.py`

---

# 🎉 Notebook Complete!

**Good luck with the hackathon!** 🚀

**IHEC-CODELAB 2.0 - La Casa del Papel Edition**